In [103]:
import requests, json, pandas as pd, ete3, tqdm, re, numpy as np, gensim, random, pickle, requests, nltk
import scipy.sparse as sp
from collections import deque
import wikipediaapi as wiki_session
from Analytical_solution_test_parallel import SetHomoplasy

In [2]:
def RandomPages(num=10):
    URL = "https://en.wikipedia.org/w/api.php"
    S = requests.Session()
    params = {'format':'json',
             'action':'query',
             'generator':'random',
             'grnnamespace':0,
             'grnlimit':num
            }
    R = S.get(url=URL, params=params)
    pages = pd.DataFrame.from_records(R.json()['query']['pages']).T.title.tolist()
    wiki = wiki_session.Wikipedia('en')
    return [wiki.page(page) for page in pages]

In [3]:
def WikiAPIlcm(page_a, page_b, max_iter=100):
    # filter
    filt_cat = ['articles', 'wiki', 'commons', 'container', 'stub', 'requiring diffusion']
    
    # might be same
    if page_a == page_b: return page_a
    
    # might be a category itself
    cat_dict = dict.fromkeys(['a', 'b'])
    cat_dict['a'] = ete3.Tree()
    cat_dict['a'].page = page_a
    cat_dict['a'].name = page_a.title 
    cat_dict['a'].queue = deque([page_a.title])
    cat_dict['b'] = ete3.Tree()
    cat_dict['b'].page = page_b
    cat_dict['b'].name = page_b.title
    cat_dict['b'].queue = deque([page_b.title])
    
    wiki = wiki_session.Wikipedia('en')
    def add_cat(tag, other_tag):
        cur_node = cat_dict[tag] & cat_dict[tag].queue.pop()
        catitems = cur_node.page.categories
        for key in catitems:
            if any(filt in key.lower() for filt in filt_cat):
                continue
            if key not in cat_dict[tag]:
                child = ete3.TreeNode(name=key)
                child.page = catitems[key]
                cur_node.add_child(child)
                cat_dict[tag].queue.append(key)
                
                if key in cat_dict[other_tag]:
                    return key
        return None
    
    def ret_path_tag(val, tag):
        path = []
        node = cat_dict[tag] & val
        while node is not None:
            path.append(node.page)
            node = node.up    
        return path[::-1]
    
    def ret_path(val):
        return (ret_path_tag(val, 'a'), ret_path_tag(val, 'b'))
    
    i = 0
    #bar = tqdm.tqdm_notebook(total=max_iter, leave=False, desc='searching lcm')
    while page_a is not None and page_b is not None and i < max_iter:
        val = add_cat('a', 'b')
        if val: return ret_path(val)
        val = add_cat('b', 'a')
        if val: return ret_path(val)
        i+=1
        #bar.update(1)
    return (None, None)

In [4]:
from itertools import combinations
from multiprocessing import Pool, cpu_count
def BuildTreeFromPages(pagelist):
    # TODO fix printing to normal aync mult
    # TODO takes for ever... 
    paths = dict.fromkeys(pagelist, [])
    pool = Pool(cpu_count())
    res = []
    for i,j in tqdm.tqdm_notebook(combinations(pagelist, 2), total=len(pagelist)*(len(pagelist)-1)//2,
                                  desc='iterating combinations'):
        res.append(((i,j), pool.apply_async(WikiAPIlcm, args=(i, j))))
    pool.close()
    pool.join()
    for (i,j), r in res:
        path_i, path_j = r.get()
        paths[i].append(path_i)
        paths[j].append(path_j)
    return paths

In [227]:
# TODO slow, bad and incomplete
pages = RandomPages(3)
BuildTreeFromPages(pages)

NameError: name 'WikiAPIlcm' is not defined

In [122]:
def WikiAPITree(topic, max_depth=2, max_child=10, max_nodes_tot=100, branch_factor=5, main_rep=True):
    """
        tree format is as follows:
        A category node is created (at root or as internal)
        Every node is aimed to be a MAIN page
        So if the category has a main page, we replace the node name with that page
        
        chechking for the existance of a main page in the tree is easy as (page.title in tree)
        
        To keep trak on used category pages, we save them at 
        
        chechking for the existance of a category page in the tree is easy as (page.title in tree.categories)
        a dict tree.categories marks the relationship between categoy and node name in tree
    """
    
    filter_pages = ['template', 'portal']
    wiki = wiki_session.Wikipedia('en')
        
    def cat_to_page(node):
        fp = requests.get(node.Catpage.fullurl)
        for l in fp.iter_lines():
            main_art = re.findall(b'mainarticle.*is.*title=\"(.*)\"', l)
            if main_art:
                main_page = wiki.page(str(main_art[0].decode('utf-8')))
                if main_page.exists():
                    node.page = main_page
                    node.pageless = False
                    node.name = node.page.title
                    break
                break
        return node
    
    t = ete3.Tree()
    t.name = topic
    t.Catpage = wiki.page('Category:'+topic if 'Category:' not in topic else topic)
    t.categories = dict()
    t.depth = 0
    t.pageless = True
    if main_rep: t = cat_to_page(t)
    t.categories[t.Catpage.title] = (t.page if not t.pageless else t.Catpage).title
    
    node_q = deque([t])
    bar = tqdm.tqdm_notebook(total=max_nodes_tot, desc='Constructing CatTree')
    node_counter = 0
    while node_q and node_counter < max_nodes_tot: # wiki category BFS
        # next category node
        node = node_q.popleft()
        
        # get potential children
        # keep it real and not alpha-betic
        subitems = node.Catpage.categorymembers
        sub_keys = list(subitems.keys())
        random.shuffle(sub_keys)
        
        # either you got a main page or you don't....
        """
            # last chance category - are you "fixable"?
            # is there a page under you with your own name?
            if node.pageless:
                if node.name in sub_keys and subitems[node.name].ns == wiki_session.Namespace.MAIN:
                    node.page = subitems[node.name]
                    node.pageless = False
                    sub_keys.remove(node.name)
                elif not keep_cat:
                    node.detach()
                    continue
        """
        
        child_count = 0
        cat_count = 0
        # adding children
        for key in sub_keys:
            # avoid circles - main and cat check
            if key in t or key in t.categories: continue 
            # filter hubs
            if any(filt in key for filt in filter_pages): continue 
            
            child = ete3.TreeNode(name=key)
            child.depth = node.depth + 1
            if subitems[key].ns == wiki_session.Namespace.CATEGORY:
                if node.depth < max_depth and cat_count < branch_factor:
                    child.Catpage = subitems[key]
                    child.name = child.Catpage.title # re.findall('Category:(.*)', key)[0]
                    child.pageless = True
                    
                    # try to replace with main_artical 'pageless' might change inside - un less in full recon mode
                    if main_rep: child = cat_to_page(child)
                    # we know the category is not in tree, but the new main article might
                    if not child.pageless and child.page.title in t: continue
                    
                    # seems like a new node
                    # when adding a new category, trak cat and main page if exists
                    t.categories[child.Catpage.title] = (child.Catpage if child.pageless else child.page).title
                    node.add_child(child) # main page name is in name already
                    node_q.append(child)
                    cat_count += 1
            elif child_count < max_child and subitems[key].ns == wiki_session.Namespace.MAIN:
                child.pageless = False
                child.page = subitems[key]
                child.name = child.page.title
                node.add_child(child)
                child_count += 1
                
        # category leaf should be erased - they are useless
        if len(node.children) < 1 and node.pageless:
            node.detach()
        else:
            node_counter += (1+child_count)
            bar.update(1+child_count)
    bar.close()
    
    # category leaf might still be in the queue
    bar = tqdm.tqdm_notebook(total=len(node_q), desc='Category Leaf Cleanup')
    while node_q:
        node = node_q.popleft()
        """
        subitems = node.Catpage.categorymembers
        if node.name in subitems and node.pageless:
            if subitems[node.name].ns == wiki_session.Namespace.MAIN:
                node.page = subitems[node.name]
                node.name = node.page.title
                node.pageless = False 
        """
        if node.pageless:
            # either you got a main page or you don't... and you don't
            parent = node.up
            node.detach()
            if (parent not in node_q and parent.pageless):
                if parent.is_leaf() and not parent.is_root():
                    # no children, the parent became a category leaf
                    node_q.append(parent)
        bar.update(1)
        
    pop = 0
    for _ in t.traverse(): pop += 1
    t.size = pop

    return t

In [123]:
from process_wikipedia import remove_special_chars, remove_html_tags, clean_string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
def text_cleanup(text):
    text = remove_html_tags(text)
    text = remove_special_chars(text, ['\n', '–']+list(punctuation))
    text = clean_string(text, set(stopwords.words('english')))
    return text

stemm = PorterStemmer()
lemma = WordNetLemmatizer()
def tokanizer(text, ret_trans=True, unique=False):
    words = nltk.word_tokenize(text)
    words, inv_words = np.unique(words, return_inverse=True)
    word_trans_ = dict.fromkeys(words)
    for i in range(len(words)): 
        word_trans_[words[i]] = stemm.stem(lemma.lemmatize(words[i]))
        words[i] = word_trans_[words[i]]
    
    tokens, inv_tokens = np.unique(words, return_inverse=True)
    if not unique: tokens = (tokens[inv_tokens])[inv_words]
    tokens = tokens.tolist()
    if ret_trans:
        return (tokens, word_trans_)
    else:
        return tokens

In [133]:
from gensim.corpora import Dictionary
from gensim import matutils
def SetWordDataDict(tree, no_below=3, full_rec=False):
    # collect tokens and translations
    token_corp = Dictionary()
    word_to_token = dict()
    for node in tqdm.tqdm_notebook(tree.traverse(), total=tree.size, desc='Collecting Page Tokens'):
        # only pages are counted
        if not node.pageless:
            node.text = text_cleanup(node.page.text)
            node.tokens, trans = tokanizer(node.text, unique=True, ret_trans=True)
            word_to_token.update(trans)
            token_corp.add_documents([node.tokens])

    # filter token space to relevant tokens
    token_corp.filter_extremes(no_below=no_below)
    
    # token_to_word inv map
    dat_size = len(token_corp)
    tree.token_corp = token_corp
    tree.word_to_token = word_to_token
    tree.token_to_word = dict()
    for k, v in word_to_token.items():
        tree.token_to_word[v] = tree.token_to_word.get(v, [])
        tree.token_to_word[v].append(k)
        
    # word2vec mapping
    for node in tqdm.tqdm_notebook(tree.traverse('postorder'), total=tree.size, desc='Token2Vec_pass1'):
        if node.pageless or (full_rec and not node.is_leaf()):
            # ancestral reconstruction
            if not node.children:
                # TODO bug check for possible cat leaf arose here
                node.detach()
                continue
            children_vecs_1 = sp.vstack([n.dat for n in node.children]).sum(axis=0)
            # indicicive positions contributed 2 for this sum, should be discounted
            indicicive = sp.vstack([(n.dat == 2) for n in node.children]).sum(axis=0)
            children_vecs_1 = children_vecs_1 - indicicive
            children_vecs_0 = len(node.children) - children_vecs_1 + indicicive
            
            cols = np.argwhere((children_vecs_1 > children_vecs_0))[:, 1]
            rows = [0]*len(cols)
            dat = [1]*len(cols)
            node.dat = sp.csr_matrix((dat, (rows, cols)), shape=(1, dat_size))
            node.dat[0, np.argwhere(children_vecs_0 == children_vecs_1)[:, 1]] = 2 # marks indicicive
        else:
            cols = np.array(token_corp.doc2idx(node.tokens))
            cols = cols[cols>-1]
            rows = [0]*len(cols)
            dat = [1]*len(cols)
            node.dat = sp.csr_matrix((dat, (rows, cols)), shape=(1, dat_size))
            
    for node in tqdm.tqdm_notebook(tree.traverse('preorder'), total=tree.size, desc='Token2Vec_pass2'):
        if node.pageless or (full_rec and not node.is_leaf()):
            indicicive = ((node.dat == 2).indices.tolist())
            if indicicive:
                if node.is_root():
                    node.dat[0, indicicive] = np.random.randint(2, size=len(indicicive))
                else:
                    node.dat[0, indicicive] = node.up.dat[0, indicicive]

In [136]:
topics = ['Physics']#, 'Finance', 'Biology', 'Mathematics', 'Computer_science']
for topic in topics:
    t = WikiAPITree(topic, max_depth=50, max_child=5, max_nodes_tot=1000, branch_factor=5)
    SetWordDataDict(t, no_below=5)
    t = SetHomoplasy(t)
    pickle.dump(t, open('../wikidat/tree_'+topic+'_with_cat.tree', 'wb'))

/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

 61%|██████    | 1025/1686 [00:00<00:00, 2495.89it/s]


In [110]:
topics = ['Physics', 'Finance', 'Biology', 'Mathematics', 'Computer_science']
for topic in topics:
    t = WikiAPITree(topic, max_depth=50, max_child=5, max_nodes_tot=500, branch_factor=5)
    SetWordDataDict(t, no_below=5)
    t = SetHomoplasy(t)
    pickle.dump(t, open('../wikidat/tree_'+topic+'_with_cat_500.tree', 'wb'))

/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

 59%|█████▉    | 552/932 [00:00<00:00, 2579.44it/s]


/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

 62%|██████▏   | 504/818 [00:00<00:00, 2690.74it/s]


/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

ValueError: blocks must be 2-D

In [135]:
topics = ['Physics', 'Finance', 'Biology', 'Mathematics', 'Computer_science']
for topic in topics:
    t = WikiAPITree(topic, max_depth=50, max_child=5, max_nodes_tot=1000, branch_factor=5, main_rep=False)
    SetWordDataDict(t, no_below=5, full_rec=True)
    t = SetHomoplasy(t)
    pickle.dump(t, open('../wikidat/tree_'+topic+'_with_cat_1000_full_rec.tree', 'wb'))

/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

 61%|██████    | 997/1646 [00:00<00:00, 2562.42it/s]


/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

...

/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

 61%|██████    | 996/1632 [00:00<00:00, 2698.97it/s]


/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/media/kishonylab/KishonyStorage/galn/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:127: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_array

 62%|██████▏   | 984/1594 [00:00<00:00, 2664.91it/s]


In [127]:
t.is_root()

True

In [132]:
np.random.randint(2, size=1)

array([0])